# 🚀 GEM-B0: 형태소 기반 ROUGE + 띄어쓰기 교정 + N-Best Reranking

**Phase 1 Quick Wins 전략:**
- 형태소 기반 ROUGE 평가
- 띄어쓰기 교정
- N-Best Reranking (ROUGE 기반 최적 후보 선택)

In [1]:
import pandas as pd
import os
import re
import json
import yaml
import random
import numpy as np
import torch
from glob import glob
from tqdm import tqdm
from pprint import pprint
from rouge import Rouge
from sklearn.feature_extraction.text import TfidfVectorizer

# Transformers & Torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    BartForConditionalGeneration, 
    BartConfig,
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
import wandb

# -----------------------------------------------------------------------------
# 1. Configuration & Seed Setting (설정 및 시드 고정)
# -----------------------------------------------------------------------------

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 고득점을 위한 하이퍼파라미터 튜닝
CONF = {
    "general": {
        "data_path": "./data/",
        "model_name": "digit82/kobart-summarization",
        "output_dir": "./results_gem_b0",
        "seed": 42
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 128,
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', 
                          '#Person5#', '#Person6#', '#Person7#', 
                          '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 15,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 32,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 3,
        "fp16": True,
        "load_best_model_at_end": True,
        "metric_for_best_model": "combined_score",
        "greater_is_better": True,
        "logging_dir": "./logs",
        "logging_strategy": "steps",
        "logging_steps": 100,
        "predict_with_generate": True,
        "generation_max_length": 128,
        "early_stopping_patience": 3,
        "report_to": "none"
    },
    "inference": {
        "ckpt_path": "./results_gem_b0/best_model",
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 3,
        "early_stopping": True,
        "generate_max_length": 128,
        "num_beams": 5,
        "length_penalty": 1.0,
        "batch_size": 32,
        "remove_tokens": [],
        # 5-1. TF-IDF 키워드 설정
        "use_tfidf_keywords": True,
        "tfidf_top_k": 5,
        # 5-2. 동적 추론 설정
        "use_dynamic_inference": True,
        "length_threshold_short": 500,    # 짧은 대화 기준 (문자 수)
        "length_threshold_long": 1500,    # 긴 대화 기준 (문자 수)
    }
}

seed_everything(CONF['general']['seed'])
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# -----------------------------------------------------------------------------
# 2. TF-IDF Keyword Extractor (5-1 구현)
# -----------------------------------------------------------------------------

class TFIDFKeywordExtractor:
    """TF-IDF 기반 키워드 추출기"""
    
    def __init__(self, corpus=None, max_features=2000):
        self.vectorizer = TfidfVectorizer(
            max_features=max_features,
            ngram_range=(1, 2),
            token_pattern=r'[가-힣]+',  # 한글만 추출
            min_df=2,
            max_df=0.9
        )
        self.is_fitted = False
        
        if corpus is not None:
            self.fit(corpus)
    
    def fit(self, corpus):
        """코퍼스로 TF-IDF 학습"""
        # 마스킹 토큰 제거 후 학습
        cleaned_corpus = [re.sub(r'#\w+#', ' ', doc) for doc in corpus]
        self.vectorizer.fit(cleaned_corpus)
        self.feature_names = self.vectorizer.get_feature_names_out()
        self.is_fitted = True
        print(f"✅ TF-IDF 학습 완료: {len(self.feature_names)}개 feature")
    
    def extract_keywords(self, text, top_k=5):
        """단일 문서에서 TF-IDF 상위 키워드 추출"""
        if not self.is_fitted:
            return []
        
        cleaned_text = re.sub(r'#\w+#', ' ', text)
        
        try:
            tfidf_vector = self.vectorizer.transform([cleaned_text])
            scores = tfidf_vector.toarray()[0]
            top_indices = scores.argsort()[-top_k:][::-1]
            
            keywords = []
            for idx in top_indices:
                if scores[idx] > 0:
                    keywords.append(self.feature_names[idx])
            
            return keywords
        except:
            return []
    
    def get_keyword_string(self, text, top_k=5):
        """키워드를 문자열로 반환"""
        keywords = self.extract_keywords(text, top_k)
        return " / ".join(keywords) if keywords else ""

# -----------------------------------------------------------------------------
# 3. Dynamic Inference Config (5-2 구현)
# -----------------------------------------------------------------------------

def get_dynamic_inference_config(dialogue_text, conf):
    """
    대화 길이에 따른 동적 추론 설정 반환
    
    짧은 대화 (< 500자): 간결한 요약
    중간 대화 (500~1500자): 표준 설정
    긴 대화 (> 1500자): 더 긴 요약, 높은 beam
    """
    length = len(dialogue_text)
    
    if length < conf['inference']['length_threshold_short']:
        # 짧은 대화
        return {
            "max_length": 80,
            "num_beams": 4,
            "length_penalty": 0.9,
            "no_repeat_ngram_size": 2,
        }
    elif length < conf['inference']['length_threshold_long']:
        # 중간 대화
        return {
            "max_length": 100,
            "num_beams": 5,
            "length_penalty": 1.0,
            "no_repeat_ngram_size": 3,
        }
    else:
        # 긴 대화
        return {
            "max_length": 128,
            "num_beams": 6,
            "length_penalty": 1.2,
            "no_repeat_ngram_size": 3,
        }

# -----------------------------------------------------------------------------
# 4. Data Preprocessing (데이터 전처리)
# -----------------------------------------------------------------------------

# Tokenizer 로드 및 Special Token 추가
tokenizer = AutoTokenizer.from_pretrained(CONF['general']['model_name'])
special_tokens_dict = {'additional_special_tokens': CONF['tokenizer']['special_tokens']}
tokenizer.add_special_tokens(special_tokens_dict)

CONF['tokenizer']['bos_token'] = tokenizer.bos_token
CONF['tokenizer']['eos_token'] = tokenizer.eos_token
CONF['inference']['remove_tokens'] = [tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token]

class Preprocess:
    def __init__(self, bos_token, eos_token):
        self.bos_token = bos_token
        self.eos_token = eos_token

    def make_set_as_df(self, file_path, is_train=True):
        df = pd.read_csv(file_path)
        if is_train:
            return df[['fname', 'dialogue', 'summary']]
        else:
            return df[['fname', 'dialogue']]

    def make_input(self, dataset, is_test=False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

class CustomDataset(Dataset):
    def __init__(self, encoder_input, decoder_input, labels=None, is_test=False, ids=None, raw_dialogues=None):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.is_test = is_test
        self.ids = ids
        self.raw_dialogues = raw_dialogues  # 5-2를 위한 원본 대화 저장

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        if 'input_ids' in self.decoder_input:
            item['decoder_input_ids'] = self.decoder_input['input_ids'][idx].clone().detach()
            item['decoder_attention_mask'] = self.decoder_input['attention_mask'][idx].clone().detach()

        if not self.is_test:
            item['labels'] = self.labels['input_ids'][idx].clone().detach()
            
        if self.ids is not None:
            item['ID'] = self.ids[idx]
        
        if self.raw_dialogues is not None:
            item['raw_dialogue'] = self.raw_dialogues[idx]
            
        return item

    def __len__(self):
        return len(self.encoder_input['input_ids'])

def prepare_data(conf, tokenizer, is_train=True):
    preprocessor = Preprocess(conf['tokenizer']['bos_token'], conf['tokenizer']['eos_token'])
    data_path = conf['general']['data_path']
    
    if is_train:
        train_df = preprocessor.make_set_as_df(os.path.join(data_path, 'train.csv'))
        val_df = preprocessor.make_set_as_df(os.path.join(data_path, 'dev.csv'))
        
        # Train Data
        enc_train, dec_in_train, dec_out_train = preprocessor.make_input(train_df)
        tokenized_enc_train = tokenizer(enc_train, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['encoder_max_len'])
        tokenized_dec_in_train = tokenizer(dec_in_train, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        tokenized_dec_out_train = tokenizer(dec_out_train, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        
        train_dataset = CustomDataset(tokenized_enc_train, tokenized_dec_in_train, tokenized_dec_out_train)
        
        # Val Data
        enc_val, dec_in_val, dec_out_val = preprocessor.make_input(val_df)
        tokenized_enc_val = tokenizer(enc_val, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['encoder_max_len'])
        tokenized_dec_in_val = tokenizer(dec_in_val, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        tokenized_dec_out_val = tokenizer(dec_out_val, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        
        val_dataset = CustomDataset(tokenized_enc_val, tokenized_dec_in_val, tokenized_dec_out_val)
        
        return train_dataset, val_dataset, train_df  # train_df 반환 추가 (TF-IDF 학습용)
    
    else: # Test
        test_df = preprocessor.make_set_as_df(os.path.join(data_path, 'test.csv'), is_train=False)
        enc_test, dec_in_test = preprocessor.make_input(test_df, is_test=True)
        
        tokenized_enc_test = tokenizer(enc_test, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['encoder_max_len'])
        tokenized_dec_in_test = tokenizer(dec_in_test, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        
        test_dataset = CustomDataset(tokenized_enc_test, tokenized_dec_in_test, is_test=True, 
                                     ids=test_df['fname'].tolist(),
                                     raw_dialogues=test_df['dialogue'].tolist())  # 원본 대화 저장
        return test_df, test_dataset

# -----------------------------------------------------------------------------
# 5. Model Training (모델 학습)
# -----------------------------------------------------------------------------
def compute_metrics(eval_pred):
    rouge = Rouge()
    predictions, labels = eval_pred
    
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions.tolist(), skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=False)
    
    remove_tokens = [tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, '<usr>']
    
    def clean_text(text_list):
        cleaned = []
        for text in text_list:
            for token in remove_tokens:
                if token is not None:
                    text = text.replace(token, "").strip() 
            cleaned.append(text)
        return cleaned

    decoded_preds = clean_text(decoded_preds)
    decoded_labels = clean_text(decoded_labels)
    
    try:
        results = rouge.get_scores(decoded_preds, decoded_labels, avg=True)
        r1 = results["rouge-1"]["f"]
        r2 = results["rouge-2"]["f"]
        rl = results["rouge-l"]["f"]
        
        combined_score = (r1 + r2 + rl) / 3
        
        return {
            "rouge-1": r1,
            "rouge-2": r2,
            "rouge-l": rl,
            "combined_score": combined_score
        }
    except Exception as e:
        return {"combined_score": 0.0}


def train():
    # 데이터 로드 (TF-IDF 학습을 위해 train_df 받기)
    train_dataset, val_dataset, train_df = prepare_data(CONF, tokenizer, is_train=True)
    
    # 5-1. TF-IDF Extractor 학습 (전체 train corpus 사용)
    print("\n>>> TF-IDF Extractor 학습 중...")
    tfidf_extractor = TFIDFKeywordExtractor(corpus=train_df['dialogue'].tolist())
    
    # TF-IDF 객체를 전역으로 저장 (추론 시 사용)
    CONF['tfidf_extractor'] = tfidf_extractor
    
    # 모델 로드
    model = BartForConditionalGeneration.from_pretrained(CONF['general']['model_name'])
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    
    # Training Arguments
    args = Seq2SeqTrainingArguments(
        output_dir=CONF['general']['output_dir'],
        overwrite_output_dir=CONF['training']['overwrite_output_dir'],
        num_train_epochs=CONF['training']['num_train_epochs'],
        learning_rate=CONF['training']['learning_rate'],
        per_device_train_batch_size=CONF['training']['per_device_train_batch_size'],
        per_device_eval_batch_size=CONF['training']['per_device_eval_batch_size'],
        warmup_ratio=CONF['training']['warmup_ratio'],
        weight_decay=CONF['training']['weight_decay'],
        lr_scheduler_type=CONF['training']['lr_scheduler_type'],
        optim=CONF['training']['optim'],
        evaluation_strategy=CONF['training']['evaluation_strategy'],
        save_strategy=CONF['training']['save_strategy'],
        save_total_limit=CONF['training']['save_total_limit'],
        fp16=CONF['training']['fp16'],
        load_best_model_at_end=CONF['training']['load_best_model_at_end'],
        metric_for_best_model=CONF['training']['metric_for_best_model'],
        greater_is_better=CONF['training']['greater_is_better'],
        logging_dir=CONF['training']['logging_dir'],
        logging_steps=CONF['training']['logging_steps'],
        predict_with_generate=CONF['training']['predict_with_generate'],
        generation_max_length=CONF['training']['generation_max_length'],
        report_to=CONF['training']['report_to'],
        seed=CONF['general']['seed']
    )
    
    # Early Stopping
    early_stopping = EarlyStoppingCallback(
        early_stopping_patience=CONF['training']['early_stopping_patience']
    )
    
    # Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping]
    )
    
    print("\n>>> Training Start...")
    trainer.train()
    
    # Best Model 저장
    best_model_path = os.path.join(CONF['general']['output_dir'], "best_model")
    trainer.save_model(best_model_path)
    print(f"\n>>> Training Finished. Best Model Saved at {best_model_path}")
    
    return best_model_path

# -----------------------------------------------------------------------------
# 6. Inference (추론) - 5-1, 5-2 적용
# -----------------------------------------------------------------------------
def inference(model_path=None):
    if model_path is None:
        model_path = os.path.join(CONF['general']['output_dir'], "best_model")
        
    print(f"\n>>> Load Model from {model_path}")
    
    model = BartForConditionalGeneration.from_pretrained(model_path)
    model.to(device)
    model.eval()
    
    # 데이터 로드
    test_df, test_dataset = prepare_data(CONF, tokenizer, is_train=False)
    
    # TF-IDF Extractor 가져오기
    tfidf_extractor = CONF.get('tfidf_extractor')
    use_tfidf = CONF['inference']['use_tfidf_keywords'] and tfidf_extractor is not None
    use_dynamic = CONF['inference']['use_dynamic_inference']
    
    print(f">>> 5-1. TF-IDF 키워드 사용: {use_tfidf}")
    print(f">>> 5-2. 동적 추론 설정 사용: {use_dynamic}")
    
    summary_list = []
    fname_list = []
    system_tokens = [tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, '<usr>']

    print("\n>>> Inference Start...")
    
    # 개별 샘플 처리 (동적 설정을 위해 배치 대신 개별 처리)
    with torch.no_grad():
        for idx in tqdm(range(len(test_dataset))):
            item = test_dataset[idx]
            input_ids = item['input_ids'].unsqueeze(0).to(device)
            attention_mask = item['attention_mask'].unsqueeze(0).to(device)
            fname = item['ID']
            raw_dialogue = item['raw_dialogue']
            
            # 5-1. TF-IDF 키워드 추출 (선택적)
            if use_tfidf:
                keywords = tfidf_extractor.get_keyword_string(raw_dialogue, top_k=CONF['inference']['tfidf_top_k'])
                if keywords:
                    # 키워드를 prefix로 추가
                    keyword_prompt = f"주요 키워드: {keywords}. "
                    keyword_ids = tokenizer.encode(keyword_prompt, add_special_tokens=False, return_tensors='pt').to(device)
                    input_ids = torch.cat([keyword_ids, input_ids[:, 1:]], dim=1)  # BOS 중복 제거
                    # Truncation
                    if input_ids.shape[1] > CONF['tokenizer']['encoder_max_len']:
                        input_ids = input_ids[:, :CONF['tokenizer']['encoder_max_len']]
                    attention_mask = torch.ones_like(input_ids)
            
            # 5-2. 동적 추론 설정
            if use_dynamic:
                gen_config = get_dynamic_inference_config(raw_dialogue, CONF)
            else:
                gen_config = {
                    "max_length": CONF['inference']['generate_max_length'],
                    "num_beams": CONF['inference']['num_beams'],
                    "length_penalty": CONF['inference']['length_penalty'],
                    "no_repeat_ngram_size": CONF['inference']['no_repeat_ngram_size'],
                }
            
            # Generate
            summary_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                early_stopping=CONF['inference']['early_stopping'],
                **gen_config
            )
            
            # Decode
            decoded = tokenizer.decode(summary_ids[0], skip_special_tokens=False)
            
            # 시스템 토큰 제거
            for token in system_tokens:
                if token is not None:
                    decoded = decoded.replace(token, "").strip()
            
            summary_list.append(decoded)
            fname_list.append(fname)
    
    # 결과 저장
    result_path = CONF['inference']['result_path']
    os.makedirs(result_path, exist_ok=True)
    
    output_df = pd.DataFrame({
        "fname": fname_list,
        "summary": summary_list
    })
    
    save_file = os.path.join(result_path, "submit_gem_b0_tfidf_dynamic.csv")
    output_df.to_csv(save_file, index=False)
    print(f"\n>>> Inference Finished. Saved at {save_file}")
    
    # 통계 출력
    lengths = [len(s) for s in summary_list]
    print(f"\n📊 요약문 통계:")
    print(f"   평균 길이: {np.mean(lengths):.1f}자")
    print(f"   최소/최대: {min(lengths)}/{max(lengths)}자")
    
    return output_df


# -----------------------------------------------------------------------------
# 7. Main Execution
# -----------------------------------------------------------------------------

if __name__ == "__main__":
    print("=" * 80)
    print("🚀 GEM-B0: TF-IDF Keywords + Dynamic Inference Strategy")
    print("=" * 80)
    
    # 1. 학습 실행
    best_ckpt = train()
    
    # 2. 추론 실행
    inference(best_ckpt)

Device: cuda:0
🚀 GEM-B0: TF-IDF Keywords + Dynamic Inference Strategy
🚀 GEM-B0: TF-IDF Keywords + Dynamic Inference Strategy

>>> TF-IDF Extractor 학습 중...

>>> TF-IDF Extractor 학습 중...
✅ TF-IDF 학습 완료: 2000개 feature
✅ TF-IDF 학습 완료: 2000개 feature


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



>>> Training Start...


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Combined Score
1,1.525000,0.595081,0.268721,0.090215,0.247860,0.202265
2,0.449200,0.526419,0.311085,0.129738,0.289494,0.243439
3,0.393900,0.513873,0.319433,0.133004,0.297505,0.249981
4,0.351900,0.510887,0.330837,0.144349,0.307782,0.260989
5,0.318800,0.514152,0.326774,0.137097,0.301571,0.255148
6,0.290200,0.522868,0.326987,0.139916,0.300172,0.255692
7,0.265500,0.530302,0.328475,0.143724,0.304351,0.258850


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



>>> Training Finished. Best Model Saved at ./results_gem_b0/best_model


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



>>> Load Model from ./results_gem_b0/best_model
>>> 5-1. TF-IDF 키워드 사용: True
>>> 5-2. 동적 추론 설정 사용: True

>>> Inference Start...
>>> 5-1. TF-IDF 키워드 사용: True
>>> 5-2. 동적 추론 설정 사용: True

>>> Inference Start...


100%|██████████| 499/499 [01:41<00:00,  4.93it/s]


>>> Inference Finished. Saved at ./prediction/submit_gem_b0_tfidf_dynamic.csv

📊 요약문 통계:
   평균 길이: 93.9자
   최소/최대: 36/216자


In [2]:
# -----------------------------------------------------------------------------
# 8. Phase 1 Quick Wins: 형태소 기반 ROUGE + 띄어쓰기 교정 + N-Best Reranking
# -----------------------------------------------------------------------------

from kiwipiepy import Kiwi

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()
print("✅ Kiwi 형태소 분석기 초기화 완료")

def tokenize_with_kiwi(text):
    """Kiwi를 사용한 형태소 분석"""
    tokens = kiwi.tokenize(text)
    return ' '.join([token.form for token in tokens])

def compute_morpheme_rouge(predictions, references):
    """형태소 단위 ROUGE 계산"""
    rouge = Rouge()
    
    # 형태소 분석
    pred_morphemes = [tokenize_with_kiwi(pred) for pred in predictions]
    ref_morphemes = [tokenize_with_kiwi(ref) for ref in references]
    
    try:
        scores = rouge.get_scores(pred_morphemes, ref_morphemes, avg=True)
        return scores
    except:
        return None

def simple_spacing_correction(text):
    """
    간단한 띄어쓰기 교정
    - 연속된 공백 제거
    - #Person# 태그 앞뒤 정리
    - 문장부호 뒤 공백 추가
    """
    # 연속 공백 제거
    text = ' '.join(text.split())
    
    # #Person# 태그 앞뒤 공백 정리
    text = re.sub(r'\s*(#Person\d+#)\s*', r' \1 ', text)
    
    # 문장부호 뒤 공백 추가
    text = re.sub(r'([.!?])\s*', r'\1 ', text)
    
    # 앞뒤 공백 제거
    text = text.strip()
    
    return text

def nbest_reranking(model, tokenizer, input_ids, attention_mask, n_best=3, reference=None):
    """
    N-Best Reranking: 여러 후보 생성 후 최적 선택
    
    Args:
        model: BART 모델
        tokenizer: 토크나이저
        input_ids: 입력 토큰 ID
        attention_mask: 어텐션 마스크
        n_best: 생성할 후보 개수
        reference: 참조 요약 (dev 셋에서만 사용)
    
    Returns:
        최적 요약문
    """
    rouge = Rouge()
    
    # 여러 후보 생성 (do_sample=True, temperature 조정)
    candidates = []
    
    for i in range(n_best):
        if i == 0:
            # 첫 번째는 기본 beam search
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                num_beams=5,
                max_length=128,
                no_repeat_ngram_size=3,
                early_stopping=True,
                length_penalty=1.0
            )
        else:
            # 나머지는 샘플링 기반
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7 + i * 0.1,  # 온도 점진적 증가
                max_length=128,
                no_repeat_ngram_size=3
            )
        
        decoded = tokenizer.decode(output_ids[0], skip_special_tokens=False)
        
        # 시스템 토큰 제거
        system_tokens = [tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, '<usr>']
        for token in system_tokens:
            if token:
                decoded = decoded.replace(token, "").strip()
        
        # 띄어쓰기 교정
        decoded = simple_spacing_correction(decoded)
        
        candidates.append(decoded)
    
    # 중복 제거
    candidates = list(set(candidates))
    
    # Reference가 있으면 ROUGE 기반 선택, 없으면 길이 기반 선택
    if reference and len(candidates) > 1:
        try:
            best_score = -1
            best_candidate = candidates[0]
            
            for cand in candidates:
                ref_morph = tokenize_with_kiwi(reference)
                cand_morph = tokenize_with_kiwi(cand)
                
                score_dict = rouge.get_scores([cand_morph], [ref_morph])[0]
                combined = (score_dict['rouge-1']['f'] + score_dict['rouge-2']['f'] + score_dict['rouge-l']['f']) / 3
                
                if combined > best_score:
                    best_score = combined
                    best_candidate = cand
            
            return best_candidate
        except:
            return candidates[0]
    else:
        # Test 셋이거나 에러 발생 시: 중간 길이 선택
        candidates.sort(key=len)
        return candidates[len(candidates) // 2]

print("✅ Phase 1 Quick Wins 함수 정의 완료")
print("   - 형태소 기반 ROUGE 평가")
print("   - 띄어쓰기 교정")
print("   - N-Best Reranking")

✅ Kiwi 형태소 분석기 초기화 완료
✅ Phase 1 Quick Wins 함수 정의 완료
   - 형태소 기반 ROUGE 평가
   - 띄어쓰기 교정
   - N-Best Reranking


In [3]:
# -----------------------------------------------------------------------------
# 9. 개선된 Inference with Phase 1 Quick Wins
# -----------------------------------------------------------------------------

def inference_with_phase1(model_path=None, use_nbest=True):
    """
    Phase 1 Quick Wins를 적용한 추론
    - N-Best Reranking
    - 형태소 기반 후처리
    - 띄어쓰기 교정
    """
    if model_path is None:
        model_path = os.path.join(CONF['general']['output_dir'], "best_model")
        
    print(f"\n{'='*80}")
    print(f">>> Load Model from {model_path}")
    print(f">>> Phase 1 Quick Wins 적용: N-Best Reranking={use_nbest}")
    print(f"{'='*80}")
    
    model = BartForConditionalGeneration.from_pretrained(model_path)
    model.to(device)
    model.eval()
    
    # 데이터 로드
    test_df, test_dataset = prepare_data(CONF, tokenizer, is_train=False)
    
    # TF-IDF Extractor 가져오기
    tfidf_extractor = CONF.get('tfidf_extractor')
    use_tfidf = CONF['inference']['use_tfidf_keywords'] and tfidf_extractor is not None
    use_dynamic = CONF['inference']['use_dynamic_inference']
    
    print(f">>> 5-1. TF-IDF 키워드 사용: {use_tfidf}")
    print(f">>> 5-2. 동적 추론 설정 사용: {use_dynamic}")
    print(f">>> Phase 1. N-Best Reranking: {use_nbest}")
    
    summary_list = []
    fname_list = []
    system_tokens = [tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, '<usr>']

    print("\n>>> Inference Start...")
    
    # 개별 샘플 처리
    with torch.no_grad():
        for idx in tqdm(range(len(test_dataset))):
            item = test_dataset[idx]
            input_ids = item['input_ids'].unsqueeze(0).to(device)
            attention_mask = item['attention_mask'].unsqueeze(0).to(device)
            fname = item['ID']
            raw_dialogue = item['raw_dialogue']
            
            # 5-1. TF-IDF 키워드 추출 (선택적)
            if use_tfidf:
                keywords = tfidf_extractor.get_keyword_string(raw_dialogue, top_k=CONF['inference']['tfidf_top_k'])
                if keywords:
                    keyword_prompt = f"주요 키워드: {keywords}. "
                    keyword_ids = tokenizer.encode(keyword_prompt, add_special_tokens=False, return_tensors='pt').to(device)
                    input_ids = torch.cat([keyword_ids, input_ids[:, 1:]], dim=1)
                    if input_ids.shape[1] > CONF['tokenizer']['encoder_max_len']:
                        input_ids = input_ids[:, :CONF['tokenizer']['encoder_max_len']]
                    attention_mask = torch.ones_like(input_ids)
            
            # 5-2. 동적 추론 설정
            if use_dynamic:
                gen_config = get_dynamic_inference_config(raw_dialogue, CONF)
            else:
                gen_config = {
                    "max_length": CONF['inference']['generate_max_length'],
                    "num_beams": CONF['inference']['num_beams'],
                    "length_penalty": CONF['inference']['length_penalty'],
                    "no_repeat_ngram_size": CONF['inference']['no_repeat_ngram_size'],
                }
            
            # Phase 1: N-Best Reranking 적용 여부
            if use_nbest:
                # N-Best 후보 생성 및 최적 선택
                summary = nbest_reranking(model, tokenizer, input_ids, attention_mask, n_best=3)
            else:
                # 기본 Generate
                summary_ids = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    early_stopping=CONF['inference']['early_stopping'],
                    **gen_config
                )
                
                decoded = tokenizer.decode(summary_ids[0], skip_special_tokens=False)
                
                # 시스템 토큰 제거
                for token in system_tokens:
                    if token is not None:
                        decoded = decoded.replace(token, "").strip()
                
                # 띄어쓰기 교정
                summary = simple_spacing_correction(decoded)
            
            summary_list.append(summary)
            fname_list.append(fname)
    
    # 결과 저장
    result_path = CONF['inference']['result_path']
    os.makedirs(result_path, exist_ok=True)
    
    output_df = pd.DataFrame({
        "fname": fname_list,
        "summary": summary_list
    })
    
    filename_suffix = "_phase1" if use_nbest else "_baseline"
    save_file = os.path.join(result_path, f"submit_gem_b0_tfidf_dynamic{filename_suffix}.csv")
    output_df.to_csv(save_file, index=False)
    print(f"\n>>> Inference Finished. Saved at {save_file}")
    
    # 통계 출력
    lengths = [len(s) for s in summary_list]
    print(f"\n📊 요약문 통계:")
    print(f"   평균 길이: {np.mean(lengths):.1f}자")
    print(f"   최소/최대: {min(lengths)}/{max(lengths)}자")
    
    # 샘플 출력
    print(f"\n📝 샘플 요약 (처음 3개):")
    for i in range(min(3, len(summary_list))):
        print(f"   [{i+1}] {summary_list[i][:100]}...")
    
    return output_df

print("✅ Phase 1 Quick Wins 적용 추론 함수 정의 완료")

✅ Phase 1 Quick Wins 적용 추론 함수 정의 완료


In [4]:
# -----------------------------------------------------------------------------
# 10. 실행: 학습 + Phase 1 Quick Wins 추론
# -----------------------------------------------------------------------------

if __name__ == "__main__":
    print("=" * 80)
    print("🚀 GEM-B0: TF-IDF + Dynamic Inference + Phase 1 Quick Wins")
    print("=" * 80)
    print("\n전략:")
    print("  1. 기본 모델 학습 (프롬프트 없이)")
    print("  2. TF-IDF 키워드 추출기 학습")
    print("  3. 추론 시 5-1, 5-2, Phase 1 적용:")
    print("     - 5-1. TF-IDF 키워드 힌트")
    print("     - 5-2. 대화 길이별 동적 추론 설정")
    print("     - Phase 1. 형태소 기반 ROUGE + 띄어쓰기 교정 + N-Best Reranking")
    print("=" * 80)
    
    # 1. 학습 실행
    print("\n[1/3] 모델 학습 시작...")
    best_ckpt = train()
    
    # 2. 기본 추론 (Phase 1 미적용)
    print("\n[2/3] 기본 추론 (Phase 1 미적용)...")
    baseline_output = inference_with_phase1(best_ckpt, use_nbest=False)
    
    # 3. Phase 1 적용 추론
    print("\n[3/3] Phase 1 Quick Wins 추론 (N-Best Reranking 적용)...")
    phase1_output = inference_with_phase1(best_ckpt, use_nbest=True)
    
    print("\n" + "=" * 80)
    print("✅ 모든 작업 완료!")
    print("=" * 80)
    print(f"\n생성된 제출 파일:")
    print(f"  1. submit_gem_b0_tfidf_dynamic_baseline.csv (Phase 1 미적용)")
    print(f"  2. submit_gem_b0_tfidf_dynamic_phase1.csv (Phase 1 적용)")
    print(f"\n💡 두 파일을 모두 리더보드에 제출하여 성능을 비교해보세요!")
    print("=" * 80)

🚀 GEM-B0: TF-IDF + Dynamic Inference + Phase 1 Quick Wins

전략:
  1. 기본 모델 학습 (프롬프트 없이)
  2. TF-IDF 키워드 추출기 학습
  3. 추론 시 5-1, 5-2, Phase 1 적용:
     - 5-1. TF-IDF 키워드 힌트
     - 5-2. 대화 길이별 동적 추론 설정
     - Phase 1. 형태소 기반 ROUGE + 띄어쓰기 교정 + N-Best Reranking

[1/3] 모델 학습 시작...

>>> TF-IDF Extractor 학습 중...

>>> TF-IDF Extractor 학습 중...
✅ TF-IDF 학습 완료: 2000개 feature
✅ TF-IDF 학습 완료: 2000개 feature


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.



>>> Training Start...


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Combined Score
1,1.502500,0.591590,0.283976,0.103554,0.263184,0.216905
2,0.449100,0.526030,0.311762,0.126670,0.287611,0.242014
3,0.393500,0.513705,0.319380,0.133945,0.294553,0.249293
4,0.351700,0.510920,0.327383,0.143527,0.302345,0.257752
5,0.318900,0.515054,0.324442,0.136187,0.297877,0.252835
6,0.290300,0.523912,0.324291,0.139964,0.295013,0.253089
7,0.265900,0.532182,0.324785,0.137562,0.300619,0.254322


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


...


[2/3] 기본 추론 (Phase 1 미적용)...

>>> Load Model from ./results_gem_b0/best_model
>>> Phase 1 Quick Wins 적용: N-Best Reranking=False
>>> 5-1. TF-IDF 키워드 사용: True
>>> 5-2. 동적 추론 설정 사용: True
>>> Phase 1. N-Best Reranking: False

>>> Inference Start...
>>> 5-1. TF-IDF 키워드 사용: True
>>> 5-2. 동적 추론 설정 사용: True
>>> Phase 1. N-Best Reranking: False

>>> Inference Start...


100%|██████████| 499/499 [01:44<00:00,  4.80it/s]
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



>>> Inference Finished. Saved at ./prediction/submit_gem_b0_tfidf_dynamic_baseline.csv

📊 요약문 통계:
   평균 길이: 96.1자
   최소/최대: 36/262자

📝 샘플 요약 (처음 3개):
   [1] Ms. Dawson은 #Person1# 에게 메시지를 전달하기 위해 사내 메모를 작성하고 배포해 달라고 요청합니다....
   [2] #Person2# 는 #Person1# 에게 출퇴근 시간에 교통체증이 심하다고 말합니다. #Person1# 은 #Person2# 에게 대중교통으로 출퇴근하는 것이 환경에도 더 좋을...
   [3] Kate는 Masha와 Hero가 이혼을 했다고 #Person1# 에게 말합니다. 그들은 서로 양육권을 가지고 있으며, 모든 것이 조용하고 원만하게 진행되고 있다고 설명합니다....

[3/3] Phase 1 Quick Wins 추론 (N-Best Reranking 적용)...

>>> Load Model from ./results_gem_b0/best_model
>>> Phase 1 Quick Wins 적용: N-Best Reranking=True
>>> 5-1. TF-IDF 키워드 사용: True
>>> 5-2. 동적 추론 설정 사용: True
>>> Phase 1. N-Best Reranking: True

>>> Inference Start...
>>> 5-1. TF-IDF 키워드 사용: True
>>> 5-2. 동적 추론 설정 사용: True
>>> Phase 1. N-Best Reranking: True

>>> Inference Start...


100%|██████████| 499/499 [04:58<00:00,  1.67it/s]


>>> Inference Finished. Saved at ./prediction/submit_gem_b0_tfidf_dynamic_phase1.csv

📊 요약문 통계:
   평균 길이: 110.6자
   최소/최대: 45/246자

📝 샘플 요약 (처음 3개):
   [1] #Person1# 은 Ms. Dawson의 메시지를 #Person2# 에게 전달합니다. #Person2# 는 메시지를 사용하는 방법을 바꾸고 메시지를 계속 사용하면 문자가 두 번 ...
   [2] #Person1# 과 출퇴근 시간이 길어지자 #Person2# 는 다른 교통수단을 제안하고, #Person1# 은 출퇴근 시간에는 대중교통을 이용하는 것이 좋을 것이라고 조언한다....
   [3] Kate는 Masha와 Hero가 이혼을 했다고 #Person1# 에게 말합니다. #Person1# 은 Kate가 조용하고 원만하게 진행되고 있으며, 집이나 주식 문제는 깔끔하게 ...

✅ 모든 작업 완료!

생성된 제출 파일:
  1. submit_gem_b0_tfidf_dynamic_baseline.csv (Phase 1 미적용)
  2. submit_gem_b0_tfidf_dynamic_phase1.csv (Phase 1 적용)

💡 두 파일을 모두 리더보드에 제출하여 성능을 비교해보세요!
